In [2]:
# Input data agent (assuming LLaMa for 3.1 8B Instruct )
import requests

url = "http://localhost:4000/v1/chat/completions"
headers = {"Content-Type": "application/json"}
data = {
    "model": "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct",
    "messages": [
        {
            "role": "system",
            "content": "You are an expert in the field of AI. Make sure to provide an explanation in few sentences."
        },
        {
            "role": "user",
            "content": "Explain the concept of AI Agent."
        }
    ],
    "stream": False,
    "max_tokens": 128
}

response = requests.post(url, headers=headers, json=data)
print(response.json())



{'id': 'chatcmpl-a7789d6942174ef1812089405dc430c8', 'object': 'chat.completion', 'created': 1752939217, 'model': '/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': 'An AI Agent is a software program that perceives its environment, makes decisions, and takes actions to achieve a specific goal or set of goals. It is a key component of Artificial Intelligence (AI) and can be thought of as a digital entity that interacts with its surroundings to accomplish tasks. \n\nAn AI Agent typically has three main components:\n\n1. **Perception**: The ability to sense and gather information about its environment, such as data from sensors, databases, or user input.\n2. **Reasoning**: The ability to process and analyze the perceived information to make decisions and determine the best course of action.\n3. **Action**:', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'length', 'stop_reason': N

In [173]:
from datetime import datetime, time

class Event:
    def __init__(self, creator: str, start_time: datetime, end_time: datetime,
                 summary: str, attendees: list[str]):
        
        # Fields initialized at the creation of the event
        self.creator = creator
        self.start_time = start_time
        self.end_time = end_time
        self.summary = summary
        self.attendees = attendees

        # Fields initialized later in the pipeline, for now set to default values
        curr_time = datetime.now()
        self.priority: float = 0.0
        self.window_start_time: datetime = datetime.combine(curr_time.date(), time(9, 0)) 
        self.window_end_time: datetime = datetime.combine(curr_time.date(), time(17, 0))
        self.final_start_time: datetime = curr_time
        self.final_end_time: datetime = curr_time

    def to_dict(self):
        return {
            'creater': self.creator,
            'start_time': self.start_time,
            'end_time': self.end_time,
            'summary': self.summary,
        }
    # Finish the rest as needed


In [29]:
def get_all_calendar_events_dummy() -> list[Event]:
    # Dummy data for testing purposes
    return [
        Event("userone.amd@gmail.com", datetime(2025, 1, 1, 10, 0, 0), 
              datetime(2025, 1, 1, 11, 0, 0), "Meeting with team", [
                  "usertwo.amd@gmail.com", "userthree.amd@gmail.com"
            ]),
        Event("usertwo.amd@gmail.com", datetime(2025, 1, 2, 12, 0, 0),
                datetime(2025, 1, 2, 13, 0, 0), "Smoke time", [
                    "userthree.amd@gmail.com"
            ])
    ]

In [22]:
events = get_all_calendar_events_dummy()


In [45]:
import requests

def set_event_priorities(events: list, url: str = "http://localhost:4000/v1/chat/completions") -> list:
    """
    Use LLM to analyze event summaries and set priority levels.
    Modifies the events in place and returns the updated list.
    
    Args:
        events: List of Event objects
        url: vLLM API endpoint URL
    
    Returns:
        List of Event objects with updated priorities
    """
    
    # Extract summaries for LLM analysis
    summaries = []
    for i, event in enumerate(events):
        summaries.append(f"Event {i+1}: {event.summary}")
    
    summaries_text = "\n".join(summaries)
    
    # System prompt for priority classification
    system_prompt = """You are an expert at analyzing calendar events and assigning priority levels.

Based on the event summary, assign one of these priority levels:
1 = FIXED (unchangeable events like appointments, deadlines, client meetings)
2 = URGENT (time-sensitive but can be slightly adjusted if needed)
3 = IMPORTANT (significant events but with some scheduling flexibility like team meetings)
4 = FLEXIBLE (can be easily rescheduled or moved)

Consider factors like:
- Keywords indicating urgency (deadline, urgent, critical, emergency)
- Meeting types (client meetings, interviews = more fixed)
- Team activities (team meetings, discussions = more flexible)
- Personal vs professional context

Return ONLY the numbers (1, 2, 3, or 4) separated by commas, nothing else.
Example output: 2, 3, 1, 4"""

    user_prompt = f"""Analyze these event summaries and assign priority levels:

{summaries_text}

Return only the priority numbers separated by commas."""

    # API request
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct",
        "messages": [
            {
                "role": "system", 
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        "stream": False,
        "max_tokens": 100,
        "temperature": 0  # Lower temperature for more consistent classification
    }
    
    try:
        # Make API request
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        
        # Extract priorities from response
        response_json = response.json()
        content = response_json['choices'][0]['message']['content'].strip()
        
        # Parse the comma-separated priorities
        priority_strings = content.split(',')
        priorities = [float(p.strip()) for p in priority_strings]
        
        # Update event objects with new priorities
        for i, event in enumerate(events):
            if i < len(priorities):
                event.priority = priorities[i]
            else:
                # Fallback if LLM didn't return enough priorities
                event.priority = 3.0  # Default to IMPORTANT
                
        return events
        
    except Exception as e:
        print(f"Error setting priorities: {e}")
        # Return events with default priorities unchanged
        return events




In [56]:
events = get_all_calendar_events_dummy()

result = set_event_priorities(events)
for i in result:
    print(i.priority, i.start_time)

3.0 2025-01-01 10:00:00
4.0 2025-01-02 12:00:00


In [53]:
##### Input AGENT

In [228]:
import requests
from datetime import datetime, time, timedelta
import json

def input_parser_agent(input_dict: dict, url: str = "http://localhost:4000/v1/chat/completions"):
    """
    Parse input dictionary and create Event object using LLM analysis.
    
    Args:
        input_dict: Dictionary containing email/meeting request data
        url: vLLM API endpoint URL
    
    Returns:
        Event object with LLM-processed fields
    """
    
    # Extract basic info (no LLM needed)
    creator = input_dict["From"]
    
    try:
        reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
    except ValueError:
        reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))

    # Parse attendees (no LLM needed)
    attendees = []
    for attendee in input_dict["Attendees"]:
        attendees.append(attendee["email"])
    
    attendees.append(input_dict["From"])
    
    # Get the day of the week for reference
    reference_day = reference_datetime.strftime("%A")
    reference_date_str = reference_datetime.strftime("%Y-%m-%d")
    
    # Calculate next few days for examples
    next_day = reference_datetime + timedelta(days=1)
    day_after_next = reference_datetime + timedelta(days=2)
    
    # Prepare LLM prompt for analysis
    system_prompt = """You are an expert at parsing meeting requests and calculating dates relative to a reference point.

CRITICAL: You MUST respond ONLY in the exact format specified below. Do not include any explanations, notes, or additional text.

DATE CALCULATION LOGIC:
The reference datetime is your anchor point. From this point, calculate the target date by counting forward:

1. RELATIVE TERMS:
   - "today" = reference date (same day)
   - "tomorrow" = reference date + 1 day
   - "next [day]" = first occurrence of that day after reference date
   - "next working day" = next Monday-Friday after reference date

2. WEEKDAY NAMES (Monday, Tuesday, etc.):
   - Always find the NEXT occurrence of that weekday from the reference date
   - If the requested day is the same as the reference day, go to next week's occurrence
   - Count forward day by day until you reach the target weekday

3. CALCULATION METHOD:
   - Start from reference date
   - Count forward until you hit the requested day
   - Never go backwards in time

TIME PARSING RULES:
- Extract specific times: "3 pm" → 15:00, "9:30 AM" → 09:30
- Extract duration: "30 minutes" → 0.5 hours, "2 hours" → 2.0 hours
- Default time if none specified: 09:00 AM
- Default duration if none specified: 1 hour
- Time windows: "morning" = 09:00-12:00, "afternoon" = 13:00-17:00, "evening" = 17:00-20:00

MANDATORY OUTPUT FORMAT (copy exactly, replace values):
START_TIME: 2025-07-24 09:00:00
END_TIME: 2025-07-24 10:00:00
SUMMARY: Project Status Meeting
PRIORITY: 3
WINDOW_START: 2025-07-24 09:00:00
WINDOW_END: 2025-07-24 18:00:00

You must provide ALL 6 lines in exactly this format with valid datetime values in YYYY-MM-DD HH:MM:SS format."""

    user_prompt = f"""REFERENCE POINT:
Datetime: {reference_datetime.strftime('%Y-%m-%d %H:%M:%S')}
Day: {reference_day}

MEETING REQUEST TO PARSE:
Subject: {input_dict["Subject"]}
Email: {input_dict["EmailContent"]}

Apply the DATE CALCULATION LOGIC: count forward from the reference point to find when the requested meeting should occur. Parse the specific time and duration from the email content.

Respond with ONLY the 6 required lines in the exact format specified."""

    # API request
    headers = {"Content-Type": "application/json"}
    data = {
            "model": "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user", 
                "content": user_prompt
            }
        ],
        "stream": False,
        "max_tokens": 300,
        "temperature": 0
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        
        # Parse LLM response
        response_json = response.json()
        content = response_json['choices'][0]['message']['content'].strip()
        
        # Extract information from LLM response with better error handling
        lines = content.split('\n')
        llm_data = {}
        
        for line in lines:
            line = line.strip()
            if line and ':' in line:
                key, value = line.split(':', 1)
                llm_data[key.strip()] = value.strip()
        
        # Validate that all required fields are present
        required_fields = ["START_TIME", "END_TIME", "SUMMARY", "PRIORITY", "WINDOW_START", "WINDOW_END"]
        missing_fields = [field for field in required_fields if field not in llm_data or not llm_data[field]]
        
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
        
        # Parse datetime strings from LLM response with validation
        try:
            start_time = datetime.strptime(llm_data["START_TIME"], "%Y-%m-%d %H:%M:%S")
            end_time = datetime.strptime(llm_data["END_TIME"], "%Y-%m-%d %H:%M:%S")
            window_start = datetime.strptime(llm_data["WINDOW_START"], "%Y-%m-%d %H:%M:%S")
            window_end = datetime.strptime(llm_data["WINDOW_END"], "%Y-%m-%d %H:%M:%S")
        except ValueError as ve:
            raise ValueError(f"Invalid datetime format in LLM response: {ve}")
            
        summary = llm_data["SUMMARY"]
        
        try:
            priority = float(llm_data["PRIORITY"])
            if priority not in [1.0, 2.0, 3.0, 4.0]:
                priority = 3.0  # Default to 3 if invalid
        except (ValueError, TypeError):
            priority = 3.0
        
    except Exception as e:
        print(f"Could not process with LLM, using fallback values. Error: {e}")
        # Fallback values if LLM call fails
        start_time = datetime.combine(reference_datetime.date(), time(9, 0))
        end_time = start_time + timedelta(hours=1)
        summary = input_dict["Subject"]
        priority = 3.0
        window_start = datetime.combine(reference_datetime.date(), time(9, 0))
        window_end = datetime.combine(reference_datetime.date(), time(18, 0))
    
    # Create Event object with priority parameter
    event = Event(creator, start_time, end_time, summary, attendees, priority + 0.5)
    
    # Set window times
    event.window_start_time = window_start
    event.window_end_time = window_end
    
    return event





--- Created Event ---
Creator: userone.amd@gmail.com
Start Time: 2025-07-25 09:00:00
End Time: 2025-07-25 10:00:00
Summary: Agentic AI Project Status Update
Attendees: ['usertwo.amd@gmail.com', 'userthree.amd@gmail.com', 'userone.amd@gmail.com']
Priority: 3.5
Window Start: 2025-07-25 09:00:00
Window End: 2025-07-25 18:00:00


In [237]:


# Example usage function
def example_usage():
    """Example of how to use the input parser agent"""
    
    sample_input = {
        "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
        "Datetime": "19-07-2025T12:34:55",
        "Location": "IISc Bangalore",
        "From": "userone.amd@gmail.com",
        "Attendees": [
            {
                "email": "usertwo.amd@gmail.com"
            },
            {
                "email": "userthree.amd@gmail.com"
            }
        ],
        "Subject": "Objectives for next financial year",
        "EmailContent": "Hey guys let hold a meet on Thursday to dicuss objectives for the upcoming financila yearat 3 pm  for 30 minutes"
    }
    
    # Create event using input parser
    event = input_parser_agent(sample_input)
    
    print(f"\n--- Created Event ---")
    print(f"Creator: {event.creator}")
    print(f"Start Time: {event.start_time}")
    print(f"End Time: {event.end_time}")
    print(f"Summary: {event.summary}")
    print(f"Attendees: {event.attendees}")
    print(f"Priority: {event.priority}")
    print(f"Window Start: {event.window_start_time}")
    print(f"Window End: {event.window_end_time}")

if __name__ == "__main__":
    example_usage()

Could not process with LLM, using fallback values. Error: 404 Client Error: Not Found for url: http://localhost:4000/v1/chat/completions

--- Created Event ---
Creator: userone.amd@gmail.com
Start Time: 2025-07-24 09:00:00
End Time: 2025-07-24 09:30:00
Summary: Objectives for next financial year
Attendees: ['usertwo.amd@gmail.com', 'userthree.amd@gmail.com', 'userone.amd@gmail.com']
Priority: 3.5
Window Start: 2025-07-24 09:00:00
Window End: 2025-07-24 18:00:00


In [200]:
reference_datetime = datetime.strptime("19-07-2025T12:34:55", "%d-%m-%YT%H:%M:%S")
reference_day = reference_datetime.strftime("%A")
print(reference_day)

Saturday


In [241]:
from datetime import datetime, timedelta
from typing import Dict

def calculate_meeting_times(ref_datetime_str: str, target_day: str) -> Dict[str, str]:
    """
    Calculates the start and end datetime for a meeting on the next
    occurrence of a given day of the week.

    Args:
        ref_datetime_str: The starting datetime as a string in the format
                          "YYYY-MM-DD HH:MM:SS".
        target_day: The desired day of the week (e.g., "monday").

    Returns:
        A dictionary with "START_TIME" and "END_TIME" strings.
    
    Raises:
        ValueError: If an invalid target_day or datetime format is provided.
    """
    
    try:
        # The format code for "YYYY-MM-DD HH:MM:SS" is "%Y-%m-%d %H:%M:%S"
        ref_datetime = datetime.strptime(ref_datetime_str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        raise ValueError("Invalid datetime format. Please use 'YYYY-MM-DD HH:MM:SS'.")

    
    days_of_week = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
        "friday": 4, "saturday": 5, "sunday": 6
    }
    
    target_day_lower = target_day.lower()
    if target_day_lower not in days_of_week:
        raise ValueError(f"Invalid target_day: '{target_day}'.")

    ref_day_num = ref_datetime.weekday()
    target_day_num = days_of_week[target_day_lower]

    days_to_add = (target_day_num - ref_day_num) % 7
    if days_to_add == 0:
        days_to_add = 7
    
    meeting_date = (ref_datetime + timedelta(days=days_to_add)).date()

    
    start_time_str = f"{meeting_date} 09:00:00"
    end_time_str = f"{meeting_date} 10:00:00"

    return {
        "START_TIME": start_time_str,
        "END_TIME": end_time_str
    }

In [242]:
# Our reference time is Monday, July 21, 2025.
reference_string = "2025-07-21 15:30:00"

# Find the meeting times for the upcoming Thursday.
meeting_times = calculate_meeting_times(reference_string, "thursday")

# Print the results in the desired format
print(f"START_TIME: {meeting_times['START_TIME']}")
print(f"END_TIME: {meeting_times['END_TIME']}")

# Expected Output:
# START_TIME: 2025-07-24 09:00:00
# END_TIME: 2025-07-24 10:00:00

START_TIME: 2025-07-24 09:00:00
END_TIME: 2025-07-24 10:00:00
